# Efficient Yelp API Calls

Kris Barbier

In [1]:
#Import json module
import json
#Open yelp_api credentials
with open('/Users/krisbarber/.secret/yelp_api.json') as f:
    login = json.load(f)
    
login.keys()

dict_keys(['client-id', 'api-key'])